# Data downloading

> Script to download LOBSTER sample data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp data_downloading

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import sys
sys.path.append("../")

In [ ]:
# | export
import io
import os
import zipfile
from typing import Literal

import click
import requests

from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import MainConfig, register_configs

In [ ]:
# | export

# access config by normal python import
cfg = MainConfig()
# register configs and then build object
register_configs()
with initialize(version_base=None, config_path=None):
    cfg_omega = compose(config_name="config")
    cfg = OmegaConf.to_object(compose(config_name="config"))
    print(cfg)
    print(cfg.universe.equities)

MainConfig(hydra=CustomHydraConf(defaults=[{'output': 'default'}, {'launcher': 'basic'}, {'sweeper': 'basic'}, {'help': 'default'}, {'hydra_help': 'default'}, {'hydra_logging': 'default'}, {'job_logging': 'default'}, {'callbacks': None}, {'env': 'default'}], mode=None, searchpath=[], run=RunDir(dir='../custom_directory/${now:%Y-%m-%d}/${now:%H-%M-%S}'), sweep=SweepDir(dir='???', subdir='???'), hydra_logging='???', job_logging='???', sweeper='???', launcher='???', callbacks={}, help=HelpConf(app_name='???', header='???', footer='???', template='???'), hydra_help=HydraHelpConf(hydra_help='???', template='???'), output_subdir='.hydra', overrides=OverridesConf(hydra=[], task=[]), job=JobConf(name='???', chdir=None, override_dirname='???', id='???', num='???', config_name='???', env_set={}, env_copy=[], config=JobConf.JobConfig(override_dirname=JobConf.JobConfig.OverrideDirname(kv_sep='=', item_sep=',', exclude_keys=[]))), runtime=RuntimeConf(version='???', version_base='???', cwd='???', co

In [ ]:
# | exports
# | code-fold: true
CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
    show_default=True,
)


@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-t", "--ticker", default=cfg.sample_data.ticker, help="ticker")
@click.option("-l", "--levels", default=cfg.sample_data.levels, help="number of levels")
def get_sample_data(
    ticker: Literal["AMZN", "AAPL", "GOOG", "INTC", "MSFT"],
    levels: Literal[1, 5, 10],
    output_dir=None,
):
    """Download and extract sample data from LOBSTER website."""
    SAMPLE_DATA_DATE = "2012-06-21"
    url = f"https://lobsterdata.com/info/sample/LOBSTER_SampleFile_{ticker}_{SAMPLE_DATA_DATE}_{levels}.zip"
    print(f"Downloading data from {url}")
    
    default_directory_name = f"data/{ticker}_{SAMPLE_DATA_DATE}_{SAMPLE_DATA_DATE}_{levels}"
    output_dir = os.path.join(os.getcwd(), default_directory_name)

    os.makedirs(output_dir, exist_ok=True)

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to download data. HTTP Status Code: {response.status_code}")
        return

    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(output_dir)

In [ ]:
show_doc(get_sample_data)


---

### <Command get-sample-data>

>      <Command get-sample-data> (*args:Any, **kwargs:Any)

Download and extract sample data from LOBSTER website.

To download and extract the [Lobster sample data](https://lobsterdata.com/info/DataSamples.php), run the following command in the directory of your choice.
```sh
$ get_sample_data
```
To download a different ticker, or different number of levels, use the -t and -l flags.
```sh
$ get_sample_data -t GOOG -l 10 
```

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()